# Input Dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

selected_country_1 = 'Indonesia'
selected_country_2 = 'Indonesia'

df1 = pd.read_csv('../input/covid19-global-dataset/worldometer_coronavirus_daily_data.csv', header =0)
df1 = df1[df1['country']== selected_country_1]
df2 = pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations.csv', header=0)
df2 = df2[df2['country']== selected_country_2]
df1.index = pd.to_datetime(df1['date'])
df2.index = pd.to_datetime(df2['date'])
print(df2)



In [ ]:
print(df1.columns)
print(df2.columns)

In [ ]:
print(np.unique(df1['country']))
print(np.unique(df2['country']))

# Analyses and Visualization

selected data to be analyzed : daily new cases, daily new death, and daily vaccination

Purpose : to show how effective vacctination to reduce daily new cases and severity of covid 19.


In [ ]:
fig,ax = plt.subplots(figsize = (10,5))
ax.plot(df1.index,df1['daily_new_cases'])
ax1 = ax.twinx ()
ax2 = ax.twinx ()
ax2.plot(df1.index,df1['daily_new_deaths'], 'r')
ax1.plot(df2.index,df2['daily_vaccinations'],'g')
plt.show()

# Improved Visualization


In [ ]:
from plotly.offline import iplot
import plotly.graph_objs as go

daily_new_cases = go.Scatter(x=df1.index, y=df1['daily_new_cases'], yaxis='y1', name='Daily confirmed cases')
daily_vaccinations = go.Scatter(x=df2.index, y=df2['daily_vaccinations'], yaxis='y2', name='Daily vaccinations')

layout_obj = go.Layout(title ='COVID vs Vaccines', xaxis=dict(title ='Date'), yaxis1=dict(title ='Daily cases'), yaxis2=dict(title='Vaccinations', side ='right', overlaying='y'))
fig = go.Figure(data=[daily_new_cases,daily_vaccinations], layout=layout_obj)
iplot(fig)
fig.write_html('outcome.html')

In [ ]:
style_object=df1.style.background_gradient(cmap='jet').highlight_max('daily_new_cases').set_caption('Daily cases')
display(style_object)

In [ ]:
df1 = pd.read_csv('../input/covid19-global-dataset/worldometer_coronavirus_daily_data.csv', header =0)
df1 = df1.groupby('country').sum()['daily_new_cases'].sort_values(ascending=False)[0:10]
df2 = pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations.csv', header=0)
df2 = df2.groupby('country').max()['people_fully_vaccinated_per_hundred'].sort_values(ascending=False)
print(df2.to_string())


#Analysis and discussion :

From the above graph we could see several fact that could confirmed some news :
1. The peak of covid 19 ocures in Indonesia was in Jan 30, 2021,which 1 month after the long New Year Holiday. 
although some interventions being implemented  to reduce the covid 19 cases, however the daily new cases were surges after the long holiday week at the end of Dec 2020. This graph confirmed, the surges of covid cases is closely related to holiday season.  

news :
https://jakartaglobe.id/news/indonesia-reports-highest-number-of-covid-deaths-in-a-day
https://jakartaglobe.id/news/indonesia-reports-highest-covid-number-in-a-day


2. Covid 19 vaccionations program has been started since 13 January 2021, by providing jabs for vurnerable population such as medical worker and eldery people. In regards to accelerate the vaccination government of Indonesia  enganged private company and public company to colloborate for the vaccination program by providing private vaccines for their employee (called as 'Vaccine Mandiri")
news : https://www.who.int/indonesia/news/detail/09-03-2021-indonesia-received-the-first-batch-of-covid-19-vaccines-from-the-covax-facility
https://www.thejakartapost.com/news/2021/05/18/private-vaccination-scheme-begins-at-jababeka.html

3. Based on data analysis daily rate for people per hundred population :  3.74, which still below the government target. From the graph above we could see the vaccination rate has dropped gradually, the slow down of vaccination amid the delayed of vaccines supplies. Developing countries are struggling to secure their vaccines supply due to some producer countries are prioritizing their supply for India

news : 
https://www.thejakartapost.com/news/2021/03/28/indonesia-slows-down-vaccine-rollout-amid-delayed-supplies.html


4. However the graph is showing how the vaccines rollout could help to intervene the raises of new covid cases in the countries. The number of daily new cases is dropped gradually.

5. the raises of covid 19 occured in late may but still below the äll time high number, the surges of covid 19 cases were triggered by Eid holiday season at mid of May 2021. 

In addition, GoI tried to do some intervention to control the covid cases by reduces the collective leave days, holidays, and ban "mudik"sesoan (the season where most of the moslem people is going back to their hometown to celeberate Eid).

news : https://go.kompas.com/read/2021/02/23/040157374/indonesia-drastically-reduces-collective-holiday-leave-days-in-2021?page=all





# Putu Trial and error code

this code are not related to the course, just trying to explore several data visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

import os
import math

from datetime import datetime, timedelta
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')

df = pd.read_csv('/kaggle/input/covid19-global-dataset/worldometer_coronavirus_daily_data.csv',
                 parse_dates=['date'], date_parser=dateparse)

df
df_summary = pd.read_csv('/kaggle/input/covid19-global-dataset/worldometer_coronavirus_summary_data.csv')

df_summary
df['continent'] = df.apply(lambda row: df_summary[df_summary.country == row.country].iloc[0].continent, axis=1)

df

fig = px.treemap(df_summary, path=["country"], values="total_confirmed", height = 750,
                 title="<b>Total Coronavirus Confirmed Cases Breakdown by Country</b>",
                 color_discrete_sequence = px.colors.qualitative.Set3)

fig.update_traces(textinfo = "label+text+value")
fig.show()

In [ ]:
def add_commas(num):
    out = ""
    counter = 0
    for n in num[::-1]:
        counter += 1
        if counter == 4:
            counter = 1
            out = "," + out
        out = n + out
    return out

print(f"As of {df.date.max().strftime('%Y-%m-%d')}, here are the numbers:\n")

print(add_commas(str(int(df_summary.total_deaths.sum()))), "total deaths. That is more than the entire population of ", end="")
deaths_ref = df_summary[df_summary.population < df_summary.total_deaths.sum()].sort_values("population", ascending=False).iloc[:2]
print(deaths_ref.iloc[0].country, f"({add_commas(str(int(deaths_ref.iloc[0].population)))}) or",
      deaths_ref.iloc[1].country, f"({add_commas(str(int(deaths_ref.iloc[1].population)))})!")

print(add_commas(str(int(df_summary.active_cases.sum()))), "active cases. You can think of that as if the entire population of ", end="")
active_ref = df_summary[df_summary.population < df_summary.active_cases.sum()].sort_values("population", ascending=False).iloc[:2]
print(active_ref.iloc[0].country, f"({add_commas(str(int(active_ref.iloc[0].population)))}), or",
      active_ref.iloc[1].country, f"({add_commas(str(int(active_ref.iloc[1].population)))}), were sick right now!")

print(add_commas(str(int(df_summary.total_recovered.sum()))), "total recoveries. It's as if the entire population of ", end="")
recover_ref = df_summary[df_summary.population < df_summary.total_recovered.sum()].sort_values("population", ascending=False).iloc[:2]
print(recover_ref.iloc[0].country, f"({add_commas(str(int(recover_ref.iloc[0].population)))}), or",
      recover_ref.iloc[1].country, f"({add_commas(str(int(recover_ref.iloc[1].population)))}), went through and recovered from Covid-19!")

In [ ]:
df_summary['log(Total Confirmed)'] = np.log2(df_summary['total_confirmed'])
df_summary['Total Confirmed'] = df_summary['total_confirmed'].apply(lambda x: add_commas(str(x)))

fig = px.choropleth(df_summary,
                    locations="country",
                    color="log(Total Confirmed)",
                    locationmode = 'country names',
                    hover_name='country',
                    hover_data=['Total Confirmed'],
                    color_continuous_scale='reds',
                    title = '<b>Coronavirus Confirmed Cases Around The Globe</b>')


log_scale_vals = list(range(0,25,2))
scale_vals = (np.exp2(log_scale_vals)).astype(int).astype(str)

scale_vals = list(map(add_commas, scale_vals))

fig.update_layout(title_font_size=22,
                  margin={"r":20, "l":30},
                  coloraxis={#"showscale":False,
                            "colorbar":dict(title="<b>Confirmed Cases</b><br>",
                                            #range=[np.log(50), np.log(6400)],
                                            titleside="top",
                                            tickmode="array",
                                            tickvals=log_scale_vals,
                                            ticktext=scale_vals
                                        )},
                 )

fig.show()